# Task 2 - Filtro de Spam Bayesiano
El objetivo es construir un clasificador de texto probabilístico desde cero.  El dataset para trabajar consiste en un texto donde cada línea es ETIQUETA \t MENSAJE, cargado desde `spam.txt`.

## 1. Pre Procesamiento

### Carga del Archivo
Primero, podemos cargar el archivo a una lista de strings para verificar que se haya leído correctamente.

In [17]:
with open('data/spam.txt', 'r') as file:
    lines = file.readlines()

for i in range(3):
    print(lines[i])

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

ham	Ok lar... Joking wif u oni...

spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's



### Limpieza de Texto
Aquí aplicamos las operaciones de conversión a minúsculas, removiendo caracteres especiales y manteniendo únicamente palabras por si solas. Por decisión de diseño, estaremos removiendo completamente los números. Adicionalmente, estaremos indagando sobre caracteres especiales y que rol pueden posiblemente jugar. Por ejemplo, un signo de dolar se "perdería" en traducción pero podríamos encodearlo a una palabra diferente.

In [ ]:
from collections import Counter

# Assuming 'lines' is your list of strings
all_text = " ".join(lines)
symbols_only = re.findall(r'[^a-zA-Z0-9\s]', all_text)
print(Counter(symbols_only).most_common())

[('.', 11042), (',', 1938), ("'", 1829), ('?', 1546), ('!', 1387), ('&', 867), (':', 738), (';', 713), ('-', 581), (')', 499), ('/', 418), ('"', 344), ('£', 329), ('*', 311), ('#', 261), ('+', 137), ('ü', 120), ('(', 119), ('Ü', 53), ('=', 47), ('\x92', 39), ('@', 38), ('‘', 37), ('|', 36), ('>', 31), ('$', 22), ('…', 16), ('_', 15), ('%', 10), ('–', 9), ('<', 6), ('[', 5), (']', 5), ('\\', 4), ('\x94', 4), ('~', 3), ('’', 3), ('\x96', 3), ('é', 3), ('“', 2), ('\x91', 2), ('\x93', 2), ('¡', 2), ('ú', 1), ('è', 1), ('^', 1), ('»', 1), ('—', 1), ('É', 1), ('ì', 1), ('鈥', 1), ('┾', 1), ('〨', 1)]


De los símbolos vistos, podríamos teorizar que los siguientes pueden tener significancia: `?`, `!`, `£`, `#`, `+`, `ü`, `ú`, `é`, `É`, `@`, `$`, `%`.

Adicionalmente, algunos de estos tienen pocas apariciones y podrían agruparse juntos; por ejemplo, `ü`, `ú`, `é`, `É` pueden agruparse como caracteres especiales.

Vamos a definir las siguientes **"clases"** para el encoding:

| Símbolo | Token (Clase) |
| :--- | :--- |
| `$`, `£`, `€` | `tagmoney` |
| `ü`, `ú`, `é`, `É` | `tagchar` |
| `!` | `tagexclamation` |
| `?` | `tagquestion` |
| `#`, `+` | `tagcontact` |
| `%` | `tagpercent` |
| `@` | `tagat` |

In [19]:
import pandas as pd
import re

processed_lines = []

# Define symbol map
symbol_map = {
    '$': ' tagmoney ',
    '£': ' tagmoney ',
    '€': ' tagmoney ',
    'ü': ' tagchar ',
    'ú': ' tagchar ',
    'é': ' tagchar ',
    'É': ' tagchar ',
    '!': ' tagexclamation ',
    '?': ' tagquestion ',
    '#': ' tagcontact ',
    '+': ' tagcontact ',
    '%': ' tagpercent ',
    '@': ' tagat '
}

# Clean every line read from file
for line in lines:

    # Replace numbers
    line = re.sub(r'\d+', ' tagnumber ', line)

    # Replace symbols
    for symbol, token in symbol_map.items():
        line = line.replace(symbol, token)
    
    # Remove any non-a-z characters & lowercase everything
    line = re.sub(r'[^a-z\s]', '', line.lower())

    # Add to list
    processed_lines.append(line)

# Print first 5
for i in range(5):
    print(processed_lines[i])

ham	go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat

ham	ok lar joking wif u oni

spam	free entry in  tagnumber  a wkly comp to win fa cup final tkts  tagnumber st may  tagnumber  text fa to  tagnumber  to receive entry questionstd txt ratetcs apply  tagnumber over tagnumber s

ham	u dun say so early hor u c already then say

ham	nah i dont think he goes to usf he lives around here though



### 3. Generación de Vocabulario
Aquí podemos simplemente utilizar list (set?) comprehension sobre cada línea, dónde separamos la línea por espacios en blanco ignorando el primer índice (el tag de spam o ham).

In [ ]:
vocab = {word for line in processed_lines for word in line.split()[1:]}
print(f"Vocabulary Size: {len(vocab)}")

Vocabulary Size: 8441
